## Method used to calculate global protected areas and OECM coverage

The protected area coverage is calculated (from WDPA [methodolgy](https://www.protectedplanet.net/en/resources/calculating-protected-area-coverage)):

1. Start with the latest WDPA monthly release
2. The WDPA is filtered to exclude records with the characteristics listed in Section 2
3. A buffer is created around protected areas reported as points using their Reported Area. There are important caveats associated with this method, some of which are explored by Visconti et al. 2013. Buffering points can underestimate or overestimate protected area coverage as the circles created around points might cover areas where protected areas do not exist (overestimation) or overlap with areas where other protected areas already exist (underestimation). It can also give inaccurate values for sites that are partly terrestrial and marine as the absence of boundaries make it difficult to predict which portion of a protected area is in the land or the sea.
4. Both polygon and buffered point layers are combined in a single layer
5. The layer above is flattened (dissolved) – to eliminate overlaps between designations and avoid double counting.
6. The global protected areas flat layer is intersected with a base map of the world (see Section 3)
7. The intersected flat layer is converted to Mollweide (an equal area projection) and the area of each polygon is calculated, in km2.
8. Calculated areas are summed by land, marine and Areas Beyond National Jurisdiction (ABNJ). Marine and coastal area are those outlined in the Economic Exclusion Zones dataset (see Section 3 above). ABNJ constitute international waters outside the 200 nautical mile limits of national jurisdiction.
9. The terrestrial protected area coverage is calculated by dividing the total area of terrestrial protected areas by total global terrestrial area excluding Antarctica. ABNJ protected area coverage is calculated by selecting areas where ISO3 = 'ABNJ'. Marine and coastal protected area coverage is total global protected areas flat coverage - (ABNJ Area + Land Area).

### Set up

In [1]:
import os
import geopandas as gpd
import pandas as pd
from datetime import datetime

In [2]:
path_in = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/raw"
path_out = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/processed"

### 1. Download and import last release of [MPA](https://www.protectedplanet.net/en/thematic-areas/marine-protected-areas): Sept 2023

In [ ]:

curl 'https://www.protectedplanet.net/downloads' \
  -H 'authority: www.protectedplanet.net' \
  -H 'accept: application/json, text/plain, */*' \
  -H 'accept-language: en,es;q=0.9,gl;q=0.8' \
  -H 'cache-control: no-cache' \
  -H 'content-type: application/json;charset=UTF-8' \
  -H 'cookie: _ProtectedPlanet_session=L0k1ZS9vaFdhbndhTkl6ZVkxd2Vza1dvNFU3NGtVbUVQNzA0SmNRTGRZNnRSMlpyeEhCNDd2czdnRkFiVFAyNUVpYzlpNFBEQmFpZG9vbjl3UlczMlRqU0ZRakRhYkJ3RmhFTy9UTWRVR3hsN1JIWnpUdTBmMWFyYVdGYVE1SWJSeWFXcnM1SlRIcUVWZk9MZ2psUXFnPT0tLWZkL2JyS0pMZmJqOUdXRlV0Z3h6OVE9PQ%3D%3D--e6ef3f79fc989a45cc1966990742d20065f88b76' \
  -H 'dnt: 1' \
  -H 'origin: https://www.protectedplanet.net' \
  -H 'pragma: no-cache' \
  -H 'referer: https://www.protectedplanet.net/en/thematic-areas/marine-protected-areas' \
  -H 'sec-ch-ua: "Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Linux"' \
  -H 'sec-fetch-dest: empty' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-site: same-origin' \
  -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36' \
  -H 'x-csrf-token: ISjsyNxbS5mkzQSQCmEzbcPltwt6SMOOV0zertwP07Lc+oIzrR9McY4JXvj17mkZ9tSd3JIoCaFfnRcC/eTWzw==' \
  --data-raw '{"domain":"general","format":"shp","token":"marine","id":21961}' \
  --compressed

In [3]:
poly1 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_0/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point1 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_0/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")
poly2 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_1/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point2 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_1/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")
poly3 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_2/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point3 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_2/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")


In [4]:
print(len(poly1))
print(len(point1))
print(len(poly2))
print(len(point2))
print(len(poly3))
print(len(point3))

6033
172
6033
172
6033
171


### 2. Filter WDPA to exclude records:
- "Non Reported" protected areas (methodology recommends to remove also "Proposed" but we keep it for future projections)
- MAB (Note: MAB sites reported as OECMs are included in coverage analyses)
- Sites submitted as points with no reported area

In [5]:
dataframes = [poly1, point1, poly2, point2, poly3, point3]

for i, df in enumerate(dataframes):
    # Remove rows where 'status' is equal to 'Not Reported'
    df = df[df['STATUS'] != 'Not Reported']

    # Remove rows where 'DESIG' contains 'MAB'
    df = df[~df['DESIG_ENG'].str.contains('MAB', case=False)]

    # Check if the dataframe is one of point1, point2, or point3
    if i in [1, 3, 5]:
        # Remove rows where reported area is 0
        df = df[(df['REP_AREA'] != 0)]

    # Update the original dataframes in the list
    dataframes[i] = df

In [6]:
print(len(dataframes[0]))
print(len(dataframes[1]))
print(len(dataframes[2]))
print(len(dataframes[3]))
print(len(dataframes[4]))
print(len(dataframes[5]))

5999
157
6018
123
6014
135


### 3. Create buffers around points based on reported area

In [7]:
# Calculate radius based on REP_AREA
def calculate_radius(rep_area):
    return (rep_area / 3.14159265358979323846) ** 0.5

# Iterate through the list and process the desired dataframes
for idx in [1, 3, 5]:
    # Get the dataframe at the specified index
    gdf = dataframes[idx]

    # Reproject in Mollweide
    gdf = gdf.to_crs('ESRI:54009')

    # Transform the reported area from square kilometers to square meters
    gdf['REP_AREA_m'] = gdf['REP_AREA'] * 1000000

    # Create the "radius" column by applying the calculate_radius function to the "REP_AREA" column
    gdf['radius'] = gdf['REP_AREA_m'].apply(calculate_radius)

    # Create buffers around the points using the "radius" column
    gdf_buffered = gdf.copy()
    gdf_buffered['geometry'] = gdf.apply(lambda row: row.geometry.buffer(row['radius']), axis=1)

    # Reproject back to WGS84
    gdf_buffered = gdf_buffered.to_crs('EPSG:4326')

    # Remove rows with invalid geometries
    gdf_buffered = gdf_buffered[gdf_buffered['geometry'].is_valid]

    # Update the original dataframe with the buffered data
    dataframes[idx] = gdf_buffered

### 4. Merge the 6 datasets (polygons and buffered points) in a single layer and segregate those that are "Proposed"

In [8]:
# Check that all of them have the same crs
first_crs = dataframes[0].crs
same_crs = all(gdf.crs == first_crs for gdf in dataframes[1:])
if same_crs:
    print("All gdf have the same crs:", first_crs)
else:
        print("gdf have different crs")

All gdf have the same crs: EPSG:4326


In [9]:
# Merge dataframes
merged_mpa = pd.concat(dataframes)
len(merged_mpa)

18445

In [ ]:
# Check if geometries are valid
sum(merged_mpa.geometry.is_valid)

In [10]:
# Save merged_mpa as a shapefile
merged_mpa.to_file(path_out + "/wdpa/merged_mpa.shp")

**Separate "Proposed"**

In [ ]:
# Select only the rows where 'STATUS' is equal to 'Proposed'
proposed = merged_mpa[merged_mpa['STATUS'] == 'Proposed']

# Select only the rows where 'STATUS' is different from 'Proposed'
protected = merged_mpa[merged_mpa['STATUS'] != 'Proposed']

In [ ]:
# Save the two dataframes as shapefiles
proposed.to_file(path_out + "/wdpa/proposed.shp")
protected.to_file(path_out + "/wdpa/protected.shp")

In [ ]:
print(len(proposed))
print(len(protected))

### 5. Dissolve intersecting polygons by relevant fields

In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
from typing import  Union, Dict
import sys
import time
from mapshaper import Mapshaper
import multiprocessing as mp
import psutil

scripts_dir = Path("..").joinpath("src")
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from pipelines.utils import load_regions
from pipelines.output_schemas import ProtectedAreaExtentSchema

In [ ]:
data = Path("../data/mpa_intermediate").resolve()
intermediate_file = data.joinpath("mpa_intermediate", "mpa_intermediate.shp").as_posix()
output_path = data.joinpath("timeseries")

output_path.mkdir(exist_ok=True, parents=True)

rename_dict = {
    "cumSumProt": "cumsum_area",
    "protectedA": "protectedAreasCount",
    "PA_DEF": "protection_status",
}
location_index = Path("../data/eez_intermediate").joinpath("locations_code.csv").as_posix()

CPU_COUNT = mp.cpu_count()
AVAILABLE_MEMORY = 64 * 1024 * 1024 * 1024  # 64 GB

In [ ]:
for year in range(2002, 2023):
    Mapshaper(16).input(files=[intermediate_file]).filter(
        expression=f"'STATUS_YR<={year}'"
    ).dissolve2(
        fields="'PA_DEF,PARENT_ISO'", calc="'protectedAreasCount = count()'"
    ).explode().reproject(
        "+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs"
    ).each(
        expression="cumSumProtectedArea=this.area/1000000"
    ).output(
        output_path.joinpath(f"protected_dissolved_{year}.shp").as_posix()
    ).execute()

### Calculate coverage statistics

In [128]:
def global_coverage_calculation(df: pd.DataFrame) -> pd.DataFrame:
    global_area = (
        df.groupby(["PA_DEF", "year", "PARENT_ISO"])
        .agg(
            {
                "cumSumProt": "sum",
                "protectedA": "first",
            }
        )
        .reset_index()
        .groupby(["PA_DEF", "year"])
        .sum()
        .reset_index()
        .assign(PARENT_ISO="GLOB")
    )
    return pd.concat(
        [
            df.replace(
                {
                    "PARENT_ISO": {
                        "COK": "NZL",
                        "IOT": "GBR",
                        "NIU": "NZL",
                        "SHN": "GBR",
                        "SJM": "NOR",
                        "UMI": "USA",
                    }
                }
            ),
            global_area,
        ],
        ignore_index=True,
    )


def separate_parent_iso(df: pd.DataFrame) -> pd.DataFrame:
    df["PARENT_ISO"] = df["PARENT_ISO"].str.split(";")
    return df.explode("PARENT_ISO")


def aggregate_area(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df.groupby(["PA_DEF", "PARENT_ISO", "year"])
        .agg({"cumSumProt": "sum", "protectedA": "first"})
        .reset_index()
    )


def add_region_iso(df: pd.DataFrame) -> pd.DataFrame:
    regions = load_regions()

    def find_region_iso(iso: str) -> Union[str, None]:
        filtered_regions = list(filter(lambda x: iso in x["country_iso_3s"], regions.get("data")))
        return filtered_regions[0]["region_iso"].strip() if len(filtered_regions) > 0 else None

    return df.assign(region=lambda row: row["PARENT_ISO"].apply(find_region_iso))


def region_coverage_calculation(df: pd.DataFrame):
    regions = (
        df.pipe(add_region_iso)
        .groupby(["PA_DEF", "region", "year"])
        .agg({"cumSumProt": "sum", "protectedA": "sum"})
        .reset_index()
    )
    return pd.concat(
        [df, regions.assign(PARENT_ISO=lambda row: row["region"]).drop(columns="region")],
        ignore_index=True,
    )


def output(df: pd.DataFrame, cols: Dict[str, str]) -> pd.DataFrame:
    locations_code = pd.read_csv(location_index, keep_default_na=False)
    return (
        df.join(locations_code.set_index("code"), on="PARENT_ISO", how="left")
        .replace({"PA_DEF": {"0": 2, "1": 1}})
        .rename(columns=cols)
        .drop(columns=["PARENT_ISO", "cumsum_area"])
        .assign(
            id=df.index + 1,
            cumSumProtectedArea=df.cumSumProt.round(2),
            protectedArea=(
                df.sort_values(by=["PARENT_ISO", "year", "PA_DEF"]).cumSumProt
                - df.sort_values(by=["PARENT_ISO", "year", "PA_DEF"])
                .groupby(["PA_DEF", "PARENT_ISO", "year"])
                .cumSumProt.shift(-1, fill_value=0)
                .reset_index(drop=True)
            ).round(2),
        )
        .set_index("id")
    )

In [132]:
years_rage = range(2000, time.localtime().tm_year)
# dissolve Mpas subtables
data_list = []
for year in years_rage:
    file = output_path.joinpath(f"protected_dissolved_{year}.shp")
    if file.exists():
        data_list.append(
            (
                gpd.read_file(file)
                .assign(year=year)
                .drop(columns=["geometry"])
                .pipe(global_coverage_calculation)
                .pipe(separate_parent_iso)
                .pipe(aggregate_area)
            )
        )
    else:
        print(f"{file} does not exist")

final = (
    pd.concat(data_list, ignore_index=True)
    .pipe(region_coverage_calculation)
    .pipe(output, rename_dict)
)

ProtectedAreaExtentSchema(final).to_csv(
    data.resolve().joinpath("protected_area_extent.csv").as_posix(),
    index=True,
)

AttributeError: 'list' object has no attribute 'joinpath'